(c) 2023 Manuel Razo. This work is licensed under a [Creative Commons
Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/).
All code contained herein is licensed under an [MIT
license](https://opensource.org/licenses/MIT).

In [1]:
# Load project package
@load_pkg Antibiotic

import Revise
# Import project package
import Antibiotic 

# Import package to handle DataFrames
import DataFrames as DF
import CSV

# Import library to list files
import Glob

# Import packages to work with data
import DataFrames as DF

# Load CairoMakie for plotting
using CairoMakie
import ColorSchemes
# Activate backend
CairoMakie.activate!()

# Set PBoC Plotting style
Antibiotic.viz.pboc_makie!()

# Data wrangling for Maeda et al., 2020 Nature Communications

In this notebook we will explore the resistance dataset from the [nature
communications](https://www.nature.com/articles/s41467-020-19713-w) paper by
Maeda et al. The main objective is to transform the data into a workable format.

## Resistance data


The data provided along with the paper consists of the changes in IC50 with
respect to the ancenstral strain for *E. coli* strains evolved in different
stresses. In other words, the entries of the table are the change in the
inhibitory concentration that reduces growth by 50% with respect to the
ancestral strain.

Let's load the data.

In [2]:
# Define data directory
data_dir = "$(git_root())/data/Maeda_2020"

# Load file into memory
df_res = CSV.read("$(data_dir)/resistance_norm.csv", DF.DataFrame)

last(df_res, 5)

Row,strain name,CP,RFP,5.FU,H2O2,CMZ,LVAL,B.Cl.Ala,AZT,KM,6.MP,AF,NiCl,CBPC,NFLX,SDC,SXZ,FOS,KTe,PS,BZ,DCS,MMC,VCM,ATP,PLM,PHEN,3.AT,NVA,MEC,TET,M9,NQO,EDTA,SHX,PMZ,NIT,SS,BSD,GAH,FTD,EM,HSE,CCCP,ABU,PUR,DVAL,5.FOA,NMNO
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5.FOAE2,-0.661645,-0.012262,-0.061384,-0.156599,-1.69766,0.095213,-0.258338,-0.722235,-1.19979,1.15146,-2.1583,1.68019,1.56062,-0.137139,-2.35551,0.378457,1.21975,3.57794,1.75876,0.339141,0.647115,-1.09392,-2.1479,-2.62838,-1.17573,-0.493887,1.06842,-2.41134,0.328002,0.499008,-0.832448,0.887298,-1.11767,-1.7816,-0.861721,-0.870322,-1.41298,0.039208,-5.23012,0.288009,-2.92426,-0.099544,-1.17567,-0.721172,-0.560523,1.53044,2.65262,-3.83195
2,NMNOE5,-0.35953,1.00274,0.634141,-0.986381,0.170903,0.157029,0.393734,0.885849,-0.373628,0.934989,-0.174868,0.790307,2.77881,-0.947897,-0.668885,-1.0005,-0.029005,0.701433,-0.31472,-0.209384,-0.692136,-0.497421,-2.69723,1.81289,-0.675218,0.361084,0.2337,1.2975,0.235668,-0.054649,-0.221308,0.505572,-0.257999,1.10787,1.67999,0.154709,1.64673,0.585146,-0.007247,0.212347,2.24802,0.672493,2.40726,0.97674,1.42789,0.918866,-0.58931,3.65591
3,NMNOE6,3.1921,2.37938,1.58008,0.363998,1.38883,0.750617,0.592988,1.06388,0.359923,-0.081964,4.89017,1.81015,0.578233,-0.690619,0.56438,2.00103,1.63801,6.04851,-0.989974,-0.029481,1.07572,-1.71851,0.966048,1.00477,0.224165,0.877591,0.466342,2.67449,-0.101146,1.02836,-0.231995,1.22119,4.0847,3.12158,0.775148,-0.115481,4.42988,0.845438,-0.070485,-0.053272,7.80305,0.076741,3.23997,1.36388,1.34384,-0.049199,3.06522,4.91633
4,NMNOE1,-2.60771,1.63187,-1.18111,-0.288041,-0.912801,0.0891,0.255163,0.763287,0.07838,0.826636,-0.23578,2.64488,0.522383,0.047844,-1.23759,-0.943455,-0.161112,0.063527,-0.135,0.100003,-0.526977,-0.412523,-3.50619,1.58498,0.085718,0.032914,-0.883043,1.26504,-1.02703,0.571841,-1.68445,-0.078044,-0.928355,-1.16824,1.61003,-1.02446,-0.731334,-1.61091,1.51119,-0.166571,-1.29776,1.05479,1.63375,0.447528,1.87663,-0.508833,-5.85575,3.55628
5,NMNOE4,2.10024,2.72564,0.115826,-1.99897,0.60069,1.37568,1.59043,-0.258301,-0.242917,-0.12508,-1.49025,-1.22764,0.015204,-1.95643,-3.45848,1.49377,1.36284,5.48681,-0.677673,-0.394002,1.11965,-7.91055,0.364236,0.927788,-4.13774,0.147863,1.94054,2.87083,-6.05579,-0.465355,0.102666,0.04573,1.18741,0.628378,1.04518,-2.59799,0.721283,-1.13886,-1.90406,-4.40247,2.69108,2.20691,3.3755,1.29044,1.4051,4.11908,1.44304,3.2644


Let's turn this table into a tidy format.

In [3]:
# Turn table into long format
df_res_tidy = DF.stack(df_res, DF.Not(Symbol("strain name")))

# Rename columns
DF.rename!(
    df_res_tidy, 
    Dict(
        Symbol("strain name") => :strain,
        :variable => :stress,
        :value => :ic50
    )
)

first(df_res_tidy, 5)

Row,strain,stress,ic50
,String15,String,Float64
1,CPE6,CP,11.7503
2,CPE2,CP,5.91574
3,CPE5,CP,2.85371
4,CPE3,CP,2.34945
5,RFPE4,CP,-0.16606


The names of the strains do not mean anything as presented. The metadata for
each of the strains can be obtained from the
[NCBI](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE137348) entry. In
there, the `GSE137348_series_matrix.txt` table contains all the metadata we
need. Specifically, we are interested in the following lines:
- `Sample_source_name_ch1`: The same names as in the table above
- `Sample_characteristics_ch1`: Several lines that contain relevant information
  about the sample.

In [4]:
# Initialize dataframe to save metadata
df_meta = DF.DataFrame()

# read lines into memory
lines_micro = readlines("$(data_dir)/GSE137348_series_matrix.txt", keep=true)

# Locate lines with strain names
idx = occursin.("Sample_source_name_ch1", lines_micro)
# Extract line with names and split by tab
strain_line = split(first(lines_micro[idx]), "\t")
# Extract strain names by removing extra characters
strain_names = [String(split(x, "\"")[2]) for x in strain_line[2:end]]
# Append values to DataFrame
df_meta.strain = strain_names

# Locate lines with strain names
idx = occursin.("Sample_characteristics_ch1", lines_micro)
# Extract lines
char_lines = lines_micro[idx]
# Loop through columns
for (i, l) in enumerate(char_lines)
    # Extract line with names and split by tab
    strain_line = split(l, "\t")
    # Extract information by removing extra characters
    strain_char = [String(split(x, "\"")[2]) for x in strain_line[2:end]]
    # Add to dataframe
    df_meta[:, Symbol("var$i")] = strain_char
end # for

first(df_meta, 5)

Row,strain,var1,var2,var3
,String,String,String,String
1,3.ATE1,genetic background: MDS42,strain type: Evolved,resistance: 3.AT resistance
2,3.ATE5,genetic background: MDS42,strain type: Evolved,resistance: 3.AT resistance
3,3.ATE3,genetic background: MDS42,strain type: Evolved,resistance: 3.AT resistance
4,3.ATE4,genetic background: MDS42,strain type: Evolved,resistance: 3.AT resistance
5,5.FOAE4,genetic background: MDS42,strain type: Evolved,resistance: 5.FOA resistance


Although the table contains the information we need, the format is too long.
Let's keep only the necessary information.

In [5]:
# Add column with genetic background
df_meta[:, :background] = [split(x, ": ")[end] for x in df_meta.var1]

# Add column for strain type
df_meta[:, :type] = [lowercase(split(x, ": ")[end]) for x in df_meta.var2]

# Add column for resistance
df_meta[:, :resistance] = [
    replace(split(x, ": ")[end], " resistance" => "") for x in df_meta.var3
]

# Remove old columns
df_meta = df_meta[:, DF.Not(["var$i" for i = 1:length(char_lines)])]

first(df_meta, 5)

Row,strain,background,type,resistance
,String,SubStrin…,String,String
1,3.ATE1,MDS42,evolved,3.AT
2,3.ATE5,MDS42,evolved,3.AT
3,3.ATE3,MDS42,evolved,3.AT
4,3.ATE4,MDS42,evolved,3.AT
5,5.FOAE4,MDS42,evolved,5.FOA


Now that we have the metadata ready, let's make sure that the strain names
overlap between the resistance table and this metadata table. For this, we'll
use the `setdiff` function to check what names are found in one but not in the
other list.

In [6]:
# Difference between resistance table and metadata table
setdiff(Set(String.(df_res_tidy.strain)), Set(df_meta.strain))

Set{String} with 4 elements:
  "CCCP3"
  "CCCP6"
  "CCCP5"
  "CCCP4"

In [7]:
# Difference between resistance table and metadata table
setdiff(Set(df_meta.strain), Set(String.(df_res_tidy.strain)))

Set{String} with 8 elements:
  "CCCPE4"
  "CCCPE5"
  "MDS42-n4"
  "MDS42-n1"
  "MDS42-n2"
  "CCCPE6"
  "CCCPE3"
  "MDS42-n3"

There are two obvious cases:
1. There is a difference in spelling between `CCCP#` and `CCCPE#`. I am not 100%
   sure these are indeed the same, but that is the most parsimonious
   explanation.
2. The resistance table is **normalized with respect to the parental strain**.
   Therefore, there are no resistance values for the `MDS42` parental strain.

Taking this into account, let's add the metadata to the resistance values
dataframe.

In [8]:
# Change names in resistance table
df_res_tidy.strain = [replace(x, "CCCP" => "CCCPE") for x in df_res_tidy.strain]

# Add metadata
DF.leftjoin!(df_res_tidy, df_meta, on=:strain)

first(df_res_tidy, 5)

Row,strain,stress,ic50,background,type,resistance
,String,String,Float64,SubStrin…?,String?,String?
1,CPE6,CP,11.7503,MDS42,evolved,CP
2,CPE2,CP,5.91574,MDS42,evolved,CP
3,CPE5,CP,2.85371,MDS42,evolved,CP
4,CPE3,CP,2.34945,MDS42,evolved,CP
5,RFPE4,CP,-0.16606,MDS42,evolved,RFP


This is the dataframe we need for our exploration. Let's save it into memory.

In [9]:
CSV.write(
    "$(git_root())/data/Maeda_2020/maeda_resistance_tidy.csv", df_res_tidy
)

"/Users/mrazo/git/antibiotic_landscape/data/Maeda_2020/maeda_resistance_tidy.csv"

## Microarray gene expression data

The other dataset provided is the gene expression data of the samples. We
already used some of the metadata information from the
[NCBI](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE137348) entry. Now, 
we will extract the actual microarray measurements into a tidy dataframe. The
gene expression data is contained between two lines `!series_matrix_table_begin`
and `!series_matrix_table_end`. Let's extract these lines.

In [48]:
# Locat indexes of lines that indicate the begin and end of the table containing
# microarray data
table_idx = findfirst(occursin.("series_matrix_table", lines_micro))

# Read table from the section indicated by the table_idx line.
df_micro = CSV.read(
    "$(data_dir)/GSE137348_series_matrix.txt", 
    DF.DataFrame; 
    skipto=table_idx + 1, 
    footerskip=1, 
    header=false
)

# Change column names to be the `ID_REF` number
DF.rename!(
    df_micro, 
    Dict(zip(names(df_micro), String.(collect(df_micro[1, :]))))
)

# Remove first row of dataframe that became redundant with column names
df_micro = df_micro[2:end, :]

# Initialize new dataframe to convert columns to floats
df_num = DF.DataFrame(; ID_REF=df_micro.ID_REF)

# Loop through columns
for name in names(df_micro[:, 2:end])
    df_num[!, name] = parse.(Float64, df_micro[:, name])
end # for

first(df_num, 5)

Row,ID_REF,GSM4076302,GSM4076304,GSM4076306,GSM4076308,GSM4076310,GSM4076311,GSM4076313,GSM4076315,GSM4076317,GSM4076319,GSM4076321,GSM4076322,GSM4076324,GSM4076325,GSM4076326,GSM4076327,GSM4076329,GSM4076330,GSM4076332,GSM4076333,GSM4076335,GSM4076337,GSM4076338,GSM4076340,GSM4076342,GSM4076344,GSM4076346,GSM4076347,GSM4076349,GSM4076351,GSM4076352,GSM4076354,GSM4076356,GSM4076357,GSM4076359,GSM4076361,GSM4076363,GSM4076364,GSM4076366,GSM4076367,GSM4076369,GSM4076371,GSM4076373,GSM4076375,GSM4076377,GSM4076379,GSM4076381,GSM4076383,GSM4076385,GSM4076387,GSM4076388,GSM4076390,GSM4076392,GSM4076394,GSM4076396,GSM4076398,GSM4076400,GSM4076402,GSM4076404,GSM4076405,GSM4076407,GSM4076409,GSM4076411,GSM4076412,GSM4076414,GSM4076416,GSM4076418,GSM4076419,GSM4076421,GSM4076423,GSM4076424,GSM4076426,GSM4076428,GSM4076430,GSM4076432,GSM4076434,GSM4076436,GSM4076438,GSM4076439,GSM4076441,GSM4076443,GSM4076445,GSM4076447,GSM4076448,GSM4076450,GSM4076451,GSM4076453,GSM4076455,GSM4076456,GSM4076458,GSM4076460,GSM4076461,GSM4076462,GSM4076464,GSM4076465,GSM4076467,GSM4076469,GSM4076471,GSM4076472,⋯
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,aaeA,1.76408,1.70065,1.60045,1.70884,1.80557,1.76934,1.68652,1.78633,1.74273,1.68652,1.66857,1.68051,1.69157,1.67051,1.6508,1.60772,1.66556,1.67353,1.64108,1.71411,1.61388,1.65623,1.75093,1.68957,1.61274,1.75192,1.69056,1.62318,1.73373,1.84361,1.90016,1.74683,2.26578,1.8827,1.7404,1.9158,1.73091,1.68957,1.7447,1.66247,1.73276,1.67353,1.71891,1.66247,1.6571,1.75807,1.74993,1.6571,1.76408,1.84252,1.77974,1.69868,1.83166,1.77881,1.68542,1.72095,1.8131,1.80962,1.84133,1.71411,1.68051,1.65519,1.71209,1.66155,1.70169,1.69157,1.73598,1.7703,1.66556,1.6571,1.77142,1.70065,1.64544,1.70591,1.67151,1.64544,1.85249,1.73494,1.73373,1.77142,1.76619,1.66857,1.76619,1.67051,1.8232,1.85359,1.7809,1.84012,1.69762,1.69255,1.70779,1.71891,1.64233,1.74893,1.65182,1.76502,1.64108,1.85027,1.67051,⋯
2,aaeB,1.66349,1.59219,1.71315,1.64108,1.68442,1.67652,1.49794,1.64652,1.60868,1.65182,1.6284,1.72201,1.62214,1.55283,1.63362,1.59416,1.60664,1.50898,1.53089,1.56255,1.6284,1.68765,1.51197,1.60045,1.47288,1.55283,1.60358,1.39121,1.70591,1.66955,1.73091,1.60254,2.39544,1.89922,1.73833,1.74273,1.58908,1.5498,1.60045,1.58306,1.66857,1.64544,1.57671,1.57573,1.59628,1.64762,1.64332,1.55479,1.66155,1.67652,1.65287,1.63362,1.67955,1.72095,1.65182,1.64869,1.69966,1.59937,1.69157,1.63148,1.6284,1.64977,1.65831,1.55775,1.4937,1.5428,1.53477,1.61487,1.585,1.58399,1.64449,1.57671,1.40698,1.60554,1.63566,1.57573,1.59726,1.58088,1.61274,1.59121,1.59121,1.62637,1.49588,1.60772,1.72786,1.71609,1.71209,1.74993,1.69347,1.69762,1.63898,1.6284,1.47914,1.69347,1.59219,1.59937,1.56874,1.90338,1.62735,⋯
3,aaeR,2.60276,2.55725,2.53946,2.58301,2.48139,2.51446,2.48059,2.65311,2.59458,2.55442,2.60676,2.59136,2.60746,2.56724,2.61731,2.56523,2.54375,2.56183,2.50739,2.5679,2.55014,2.57186,2.56451,2.57381,2.59398,2.5445,2.58242,2.57711,2.57652,2.47458,2.57381,2.53152,2.54948,2.59074,2.53728,2.51194,2.60147,2.60087,2.58037,2.6235,2.5205,2.60676,2.62966,2.6205,2.48279,2.51667,2.49251,2.52345,2.68619,2.72275,2.66703,2.61474,2.60276,2.58675,2.61049,2.58242,2.53584,2.50814,2.50598,2.46148,2.59265,2.54233,2.59638,2.53946

Having read the information, let's transform this to a tidy dataframe.

In [57]:
# Transform microarray data to a tidy format
df_micro_tidy = DF.stack(df_num, DF.Not(:ID_REF))
# Rename columns
DF.rename!(df_micro_tidy, :variable => :geo_accession, :ID_REF => :gene)

first(df_micro_tidy, 5)

Row,gene,geo_accession,value
,String15,String,Float64
1,aaeA,GSM4076302,1.76408
2,aaeB,GSM4076302,1.66349
3,aaeR,GSM4076302,2.60276
4,aaeX,GSM4076302,2.1609
5,aas,GSM4076302,2.68559


To add the corresponding data, we need to add the connecting column to our
`df_meta` dataframe. This column is listed in the text file as
`!Sample_geo_accession`.

In [58]:
# Extract line with sample_geo_accession
l = first(lines_micro[occursin.("!Sample_geo_accession", lines_micro)])

# Extract line with names and split by tab
strain_line = split(l, "\t")
# Extract information by removing extra characters
strain_char = [String(split(x, "\"")[2]) for x in strain_line[2:end]]

# Add column to metadata dataframe
df_meta[!, :geo_accession] = strain_char

first(df_meta, 5)

Row,strain,background,type,resistance,geo_accession
,String,SubStrin…,String,String,String
1,3.ATE1,MDS42,evolved,3.AT,GSM4076302
2,3.ATE5,MDS42,evolved,3.AT,GSM4076304
3,3.ATE3,MDS42,evolved,3.AT,GSM4076306
4,3.ATE4,MDS42,evolved,3.AT,GSM4076308
5,5.FOAE4,MDS42,evolved,5.FOA,GSM4076310


Having added the information, we can append this metadata to our expression
value.

In [59]:
# Add metadata to dataframe
DF.leftjoin!(df_micro_tidy, df_meta, on=:geo_accession)

first(df_micro_tidy, 5)

Row,gene,geo_accession,value,strain,background,type,resistance
,String15,String,Float64,String?,SubStrin…?,String?,String?
1,aaeA,GSM4076302,1.76408,3.ATE1,MDS42,evolved,3.AT
2,aaeB,GSM4076302,1.66349,3.ATE1,MDS42,evolved,3.AT
3,aaeR,GSM4076302,2.60276,3.ATE1,MDS42,evolved,3.AT
4,aaeX,GSM4076302,2.1609,3.ATE1,MDS42,evolved,3.AT
5,aas,GSM4076302,2.68559,3.ATE1,MDS42,evolved,3.AT


Excellent! We have the required table! Let's save it.

In [60]:
CSV.write(
    "$(git_root())/data/Maeda_2020/maeda_microarray_tidy.csv", df_micro_tidy
)

"/Users/mrazo/git/antibiotic_landscape/data/Maeda_2020/maeda_microarray_tidy.csv"